# HiC data scaffolding

* map HiC short reads to scaffolds and send bam to Ali for manually scaffolding
* also trying HiC scaffolding tools such as ALLHIC

## HiC data

In [5]:
HICR1_RAW=/workspace/hraczw/github/GA/Gillenia_genome/002.Fastp.trimming/R1.cleaned.specifiedAdapter.short.Q15.fq.gz
HICR2_RAW=/workspace/hraczw/github/GA/Gillenia_genome/002.Fastp.trimming/R2.cleaned.specifiedAdapter.short.Q15.fq.gz

## Assembly 
* This assembly was generated using Shasta with 2 iterations of PILON correction

In [13]:
ASSEMBLY_LINKS_I8=/workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling/scaff_links_i8_gapFilled.noCorrection.tgs-gapcloser.scaff_seqs.nobacteria.fasta
ASSEMBLY_LINKS_I6=/workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling/scaff_links_i6_gapFilled.noCorrection.tgs-gapcloser.scaff_seqs.nobacteria.fasta

In [1]:
ASSEMBLY_RAGOO=/powerplant/workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling/scaff_ragoo_gapfilled_noContamination.fasta

In [1]:
ASSEMBLY_RAGOO_PLUSTELO=/workspace/hraczw/github/GA/Gillenia_genome/011.RagooScaffolding/scaff_ragoo_gapfilled_noContamination.plusFlyeTelo.fasta

In [3]:
perl /workspace/hraczw/github/programs/prinseq-lite-0.20.4/prinseq-lite.pl -fasta $ASSEMBLY_LINKS_I8 -min_len 10000 -out_good \
/workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling/scaff_links_i8_gapFilled.noCorrection.tgs-gapcloser.scaff_seqs.nobacteria.ml10000

-p destination: Broken pipe
Input and filter stats:
	Input sequences: 4,077
	Input bases: 296,273,741
	Input mean length: 72669.55
	Good sequences: 779 (19.11%)
	Good bases: 281,539,680
	Good mean length: 361411.66
	Bad sequences: 3,298 (80.89%)
	Bad bases: 14,734,061
	Bad mean length: 4467.57
	Sequences filtered by specified parameters:
	min_len: 3298


In [4]:
perl /workspace/hraczw/github/programs/prinseq-lite-0.20.4/prinseq-lite.pl -fasta $ASSEMBLY_LINKS_I6 -min_len 10000 -out_good \
/workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling/scaff_links_i6_gapFilled.noCorrection.tgs-gapcloser.scaff_seqs.nobacteria.ml10000

-p destination: Broken pipe
Input and filter stats:
	Input sequences: 4,160
	Input bases: 294,655,214
	Input mean length: 70830.58
	Good sequences: 804 (19.33%)
	Good bases: 279,620,331
	Good mean length: 347786.48
	Bad sequences: 3,356 (80.67%)
	Bad bases: 15,034,883
	Bad mean length: 4480.00
	Sequences filtered by specified parameters:
	min_len: 3356


In [7]:
ASSEMBLY_LINKS_I8_ML10000=/workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling/scaff_links_i8_gapFilled.noCorrection.tgs-gapcloser.scaff_seqs.nobacteria.ml10000.fasta
ASSEMBLY_LINKS_I6_ML10000=/workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling/scaff_links_i6_gapFilled.noCorrection.tgs-gapcloser.scaff_seqs.nobacteria.ml10000.fasta

In [2]:
CHR=/output/genomic/plant/Gillenia/trifoliata/Assembly/gillenia_genome_v1.NCBI.fasta

## 1. Mapping - following in steps on ALLHIC wiki

In [3]:
module load bwa
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.1
  2) texlive/20151117   5) perlbrew/0.76      8) bwa/0.7.17
  3) pandoc/1.19.2      6) perl/5.28.0


In [5]:
mkdir 008.Allhic.scaffolding

In [1]:
WORKDIR=/workspace/hraczw/github/GA/Gillenia_genome/008.Allhic.scaffolding

In [5]:
bsub -J bwa \
-o $WORKDIR/bwa_index_ragoo.out \
-e $WORKDIR/bwa_index_ragoo.err \
"bwa index $ASSEMBLY_RAGOO"

Job <266749> is submitted to default queue <lowpriority>.


In [9]:
bsub -J bwa \
-o $WORKDIR/bwa_index_i8.out \
-e $WORKDIR/bwa_index_i8.err \
"bwa index $ASSEMBLY_LINKS_I8_ML10000"

Job <239471> is submitted to default queue <lowpriority>.


In [7]:
bsub -J bwa \
-o $WORKDIR/bwa_index_ragoo_plustelo.out \
-e $WORKDIR/bwa_index_ragoo_plustelo.err \
"bwa index $ASSEMBLY_RAGOO_PLUSTELO"

Job <403470> is submitted to default queue <lowpriority>.


In [4]:
bsub -J bwa \
-o $WORKDIR/bwa_index_chrs.out \
-e $WORKDIR/bwa_index_chrs.err \
"bwa index $CHR"

Job <587421> is submitted to default queue <lowpriority>.


In [11]:
module load samtools
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.1
  2) texlive/20151117   5) perlbrew/0.76      8) bwa/0.7.17
  3) pandoc/1.19.2      6) perl/5.28.0        9) samtools/1.9


In [6]:
bsub -J map \
-n 40 \
-o $WORKDIR/map_ragoo.out \
-e $WORKDIR/map_ragoo.err \
"bwa mem \
-t 40 \
$ASSEMBLY_RAGOO \
$HICR1_RAW \
$HICR2_RAW \
-o $WORKDIR/mapped_ragoo.sam"

Job <266750> is submitted to default queue <lowpriority>.


In [15]:
bsub -J map \
-n 40 \
-o $WORKDIR/map_i8.out \
-e $WORKDIR/map_i8.err \
"bwa mem \
-t 40 \
$ASSEMBLY_LINKS_I8_ML10000 \
$HICR1_RAW \
$HICR2_RAW \
-o $WORKDIR/mapped_i8.sam"

Job <239985> is submitted to default queue <lowpriority>.


In [8]:
bsub -J map \
-n 40 \
-o $WORKDIR/map_ragoo_plustelo.out \
-e $WORKDIR/map_ragoo_plustelo.err \
"bwa mem \
-t 40 \
$ASSEMBLY_RAGOO_PLUSTELO \
$HICR1_RAW \
$HICR2_RAW \
-o $WORKDIR/mapped_ragoo_plustelo.sam"

Job <405868> is submitted to default queue <lowpriority>.


In [6]:
bsub -J map \
-n 40 \
-o $WORKDIR/map_ragoo_chrs.out \
-e $WORKDIR/map_ragoo_chrs.err \
"bwa mem \
-t 40 \
$CHR \
$HICR1_RAW \
$HICR2_RAW \
-o $WORKDIR/mapped_ragoo_chrs.sam"

Job <587966> is submitted to default queue <lowpriority>.


In [4]:
export PATH=/workspace/hraczw/github/programs/ALLHiC/bin/:$PATH

In [5]:
export PATH=/workspace/hraczw/github/programs/ALLHiC/scripts/:$PATH

In [6]:
ls /workspace/hraczw/github/programs/ALLHiC/scripts

ALLHiC2ALLMAPS.pl  filterBAM_forHiC.pl	       partition.pl
bam2CLM.pl	   getFalen.pl		       PreprocessSAMs.pl
bam2CLM_simple.pl  gmap2AlleleTable.pl	       prune.pl
bam2net.pl	   link_superscaffold.pl       release3DDNA.pl
bam_HiCplotter.py  make_bed_around_RE_site.pl  remove_reads.pl
blastn_parse.pl    mc_bam.pl		       simuCTG.pl
classify.pl	   partition_gmap.pl	       statAGP.pl


In [7]:
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.1
  2) texlive/20151117   5) perlbrew/0.76      8) bwa/0.7.17
  3) pandoc/1.19.2      6) perl/5.28.0


In [8]:
module load bedtools
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.1
  2) texlive/20151117   5) perlbrew/0.76      8) bwa/0.7.17
  3) pandoc/1.19.2      6) perl/5.28.0        9) bedtools/2.27.1


In [9]:
module load samtools
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    5) perlbrew/0.76      9) bedtools/2.27.1
  2) texlive/20151117   6) perl/5.28.0       10) samtools/1.9
  3) pandoc/1.19.2      7) asub/2.1
  4) git/2.21.0         8) bwa/0.7.17


In [10]:
PreprocessSAMs.pl


PreprocessSAMs.pl: A script to prepare SAM or BAM files for use with Lachesis.

Syntax: /workspace/hraczw/github/programs/ALLHiC/scripts/PreprocessSAMs.pl <sam-or-bam-filename> <draft-assembly-fasta> enzyme(HINDIII/MBOI)



In [16]:
bsub -J preprocess \
-o $WORKDIR/preprocess_ragoo.out \
-e $WORKDIR/preprocess_ragoo.err \
"PreprocessSAMs.pl $WORKDIR/mapped_ragoo.sam $ASSEMBLY_RAGOO MBOI"

Job <266762> is submitted to default queue <lowpriority>.


In [25]:
bsub -J preprocess \
-o $WORKDIR/preprocess_i8.out \
-e $WORKDIR/preprocess_i8.err \
"PreprocessSAMs.pl $WORKDIR/mapped_i8.sam $ASSEMBLY_LINKS_I8 MBOI"

Job <240010> is submitted to default queue <lowpriority>.


In [16]:
bsub -J preprocess \
-o $WORKDIR/preprocess_ragoo_plustelo.out \
-e $WORKDIR/preprocess_ragoo_plustelo.err \
"PreprocessSAMs.pl $WORKDIR/mapped_ragoo_plustelo.sam $ASSEMBLY_RAGOO_PLUSTELO MBOI"

Job <406871> is submitted to default queue <lowpriority>.


In [12]:
bsub -J preprocess \
-o $WORKDIR/preprocess_ragoo_chr.out \
-e $WORKDIR/preprocess_ragoo_chr.err \
"PreprocessSAMs.pl $WORKDIR/mapped_ragoo_chrs.sam $CHR MBOI"

Job <636818> is submitted to default queue <lowpriority>.


In [17]:
rm $WORKDIR/mapped_i6.sam $WORKDIR/mapped_i8.sam

### 1.1 trying to filter multiply mapped reads

In [16]:
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.1
  2) texlive/20151117   5) perlbrew/0.76      8) bwa/0.7.17
  3) pandoc/1.19.2      6) perl/5.28.0        9) samtools/1.9


In [12]:
bsub -J filter \
-o $WORKDIR/filter_multiplyMapped_i6.out \
-e $WORKDIR/filter_multiplyMapped_i6.err \
"samtools view -h $WORKDIR/mapped_i6.bam | grep -v -e 'XA:Z:' -e 'SA:Z:' | samtools view -b > $WORKDIR/unique_mapped_i6.bam"

Job <234874> is submitted to default queue <lowpriority>.


In [13]:
bsub -J preprocess \
-o $WORKDIR/preprocess_unique.out \
-e $WORKDIR/preprocess_unique.err \
"PreprocessSAMs.pl $WORKDIR/unique_mapped.bam $ASSEMBLY_ML10000 MBOI"

Job <234876> is submitted to default queue <lowpriority>.


## 2. ALLHIC scaffolding

In [18]:
ALLHiC_partition -h

Unknown option: h
************************************************************************
    Usage: ALLHiC_partition -r draft.asm.fasta -e enzyme_sites -k Num of groups
      -h : help and usage.
      -b : prunned bam (optional, default prunning.bam)
      -r : draft.sam.fasta
      -e : enzyme_sites (HindIII: AAGCTT; MboI: GATC)
      -k : number of groups (user defined K value)
      -m : minimum number of restriction sites (default, 25)
************************************************************************


: 255

In [17]:
cd $WORKDIR/ALLHIC_ragoo_noBac
ls

In [19]:
bsub -J partition \
-o partition.out \
-e partition.out \
"ALLHiC_partition -b mapped_ragoo.REduced.paired_only.bam -r $ASSEMBLY_RAGOO -e GATC -k 9"

Job <266847> is submitted to default queue <lowpriority>.


In [20]:
bsub -J extract \
-o extract.out \
-e extract.err \
"allhic extract mapped_ragoo.REduced.paired_only.bam $ASSEMBLY_RAGOO --RE GATC"

Job <266851> is submitted to default queue <lowpriority>.


In [21]:
for i in {1..9}; \
do bsub -J allhicOpt -o allhicOpt_g${i}.out -e allhicOpt_g${i}.err \
"allhic optimize mapped_ragoo.REduced.paired_only.counts_GATC.9g$i.txt \
mapped_ragoo.REduced.paired_only.clm"; done

Job <266852> is submitted to default queue <lowpriority>.
Job <266853> is submitted to default queue <lowpriority>.
Job <266854> is submitted to default queue <lowpriority>.
Job <266855> is submitted to default queue <lowpriority>.
Job <266856> is submitted to default queue <lowpriority>.
Job <266857> is submitted to default queue <lowpriority>.
Job <266858> is submitted to default queue <lowpriority>.
Job <266859> is submitted to default queue <lowpriority>.
Job <266860> is submitted to default queue <lowpriority>.


In [22]:
ALLHiC_build $ASSEMBLY_RAGOO

1. tour format to agp ...
Processing mapped_ragoo.REduced.paired_only.counts_GATC.9g1.tour ...
Processing mapped_ragoo.REduced.paired_only.counts_GATC.9g2.tour ...
Processing mapped_ragoo.REduced.paired_only.counts_GATC.9g3.tour ...
Processing mapped_ragoo.REduced.paired_only.counts_GATC.9g4.tour ...
Processing mapped_ragoo.REduced.paired_only.counts_GATC.9g5.tour ...
Processing mapped_ragoo.REduced.paired_only.counts_GATC.9g6.tour ...
Processing mapped_ragoo.REduced.paired_only.counts_GATC.9g7.tour ...
Processing mapped_ragoo.REduced.paired_only.counts_GATC.9g8.tour ...
Processing mapped_ragoo.REduced.paired_only.counts_GATC.9g9.tour ...


In [23]:
perl /workspace/hraczw/github/programs/ALLHiC/scripts/getFalen.pl \
-i groups.asm.fasta \
-o len.txt

In [26]:
grep 'mapped_ragoo.REduced.paired_only.counts_GATC' len.txt > chrn.list

In [27]:
cat chrn.list

mapped_ragoo.REduced.paired_only.counts_GATC.9g1	226375874
mapped_ragoo.REduced.paired_only.counts_GATC.9g2	27914923
mapped_ragoo.REduced.paired_only.counts_GATC.9g3	11085632
mapped_ragoo.REduced.paired_only.counts_GATC.9g4	11057837
mapped_ragoo.REduced.paired_only.counts_GATC.9g5	8849564
mapped_ragoo.REduced.paired_only.counts_GATC.9g6	1818773
mapped_ragoo.REduced.paired_only.counts_GATC.9g7	1645931
mapped_ragoo.REduced.paired_only.counts_GATC.9g8	1615206
mapped_ragoo.REduced.paired_only.counts_GATC.9g9	47180


In [28]:
module load pfr-python3/3.6.6

In [29]:
module list

Currently Loaded Modulefiles:
  1) powerPlant/core     5) perlbrew/0.76       9) bedtools/2.27.1
  2) texlive/20151117    6) perl/5.28.0        10) samtools/1.9
  3) pandoc/1.19.2       7) asub/2.1           11) pfr-python3/3.6.6
  4) git/2.21.0          8) bwa/0.7.17


In [31]:
bsub -J plot \
-o plot_100k.out \
-e plot_100k.err \
"ALLHiC_plot mapped_ragoo.REduced.paired_only.bam groups.agp chrn.list 100k pdf"

Job <266907> is submitted to default queue <lowpriority>.


In [48]:
ls

allhicOpt_g10.err   mapped.REduced.paired_only.bam
allhicOpt_g10.out   mapped.REduced.paired_only.clm
allhicOpt_g11.err   mapped.REduced.paired_only.clusters.txt
allhicOpt_g11.out   mapped.REduced.paired_only.counts_GATC.12g10.tour
allhicOpt_g12.err   mapped.REduced.paired_only.counts_GATC.12g10.txt
allhicOpt_g12.out   mapped.REduced.paired_only.counts_GATC.12g11.tour
allhicOpt_g1.err    mapped.REduced.paired_only.counts_GATC.12g11.txt
allhicOpt_g1.out    mapped.REduced.paired_only.counts_GATC.12g12.tour
allhicOpt_g2.err    mapped.REduced.paired_only.counts_GATC.12g12.txt
allhicOpt_g2.out    mapped.REduced.paired_only.counts_GATC.12g1.tour
allhicOpt_g3.err    mapped.REduced.paired_only.counts_GATC.12g1.txt
allhicOpt_g3.out    mapped.REduced.paired_only.counts_GATC.12g2.tour
allhicOpt_g4.err    mapped.REduced.paired_only.counts_GATC.12g2.txt
allhicOpt_g4.out    mapped.REduced.paired_only.counts_GATC.12g3.tour
allhicOpt_g5.err    mapped.REduced.paired_only.counts_GATC.12g3.txt
allhicOpt_